In [1]:
import csv
from google.colab import files
import torch
import torch.nn as nn
import re

# Task 1: Name Entity Recognition
## Step 1:
We start by loading the sample.txt file provided. Then we want to seperate the different news pieces or social media comments from each other. We do this by recognizing that most of the times, whenever a new comment or news piece is written in the txt file, it is started in a new line. Therefore, we consider that a new news piece or comment is being started whever we come across the new line character. We recognize that we need to do this if we need to count the number of articles that mention gold and silver. Also, this would be useful in solving other parts of the problem. An important note to remember for the rest of the markdown file is that whenever the word article is referred to in the rest of the file, it means a particular instance of a news piece or a comment taken from the txt file that has been given.

In [2]:
#We start by loading the sample.txt file provided. Then we want to seperate the different news pieces of social media comments from each other. We do this by recognizing that most of the times, whenever a new comment or news piece is written in the txt file, it is started in a new line. Therefore, we spereate the
# Open the file in read mode
with open('sample.txt', 'r') as file:
    # Read the entire contents of the file
    file_contents = file.read()

# Seperate the different news pieces and social media articles from each other.
articles = file_contents.split('\n')
print(len(articles))

1870


In [3]:
print(articles[0])

@fansoniclove Gold the Tenrec


##Step 2:
To find out whether an article contains gold or silver, we make a function. This function takes an aritcle. Then it seperates the article into words. The method that we use to seperate the article into words disregards things like capital letters and punctuation marks. Therefore, it prevents gold!, Gold and gold from being considered as seperate words for example. It convers all the words into lower case for ease while mathcing words. Then we just check if the words list obtained from the article contains 'gold' or 'silver'. We can be confident that 'gold' and 'silver' would probably contain most instances of the two words because of the preprocessing we did.

In [4]:
#Function that takes in an article and tells if the article contains the word 'silver' or 'gold'.
def string_matcher(article):
  words = re.findall(r'\b\w+\b', article.lower())                 #This helps seperate the article into unique words. It ignores punctuation and capital letters so Gold and gold are cosidered the same word and appear 1 time.
  bool_sil = 0
  bool_gol = 0
  for i in range(len(words)):
    if words[i].lower()=='silver':
      bool_sil = 1
    if words[i].lower()=='gold':
      bool_gol = 1
  return bool_sil,bool_gol


In [5]:
#Counts the total articles containing silver and the total articles containing gold and prints them.
total_silver = 0
total_gold = 0
for i in range(len(articles)):
  sil,gol = string_matcher(articles[i])
  total_silver += sil
  total_gold += gol
print('Total number of articles containing the word silver are: ',total_silver)
print('Total number of articles containing the word gold are: ', total_gold)

Total number of articles containing the word silver are:  100
Total number of articles containing the word gold are:  605


## Step 3 Accounting for False Positives:
Along with the example of 'silver lining' give in the question, an example of an instance of false positive is sentences where phrases like 'gold medal' are used. Along with wanting to disregard these instances, we also notice that there are several commonalities in false positives that we need to teach our program to disregard. An example is phrases of the form 'as gold/silver' or 'like gold/silver'. This is because in these contexts, gold and silver are probably not being used in their literal meaning but are used as a metaphor or a simile. An example sentence from everyday usage can be, 'Knowledge is as valuable as gold these days'. Here, clearly, gold is a false positive. This shows how our method of excluding these instances can be generalized to other sentences from everyday language. Since we are searching for the words 'gold' or 'silver', our previously used preprocessing technique would already ensure we do not count gold when it is in company names (for example Goldman Sachs because Goldman is not counted as gold). Finally, another thing that leads to false positives can be using these words while referring to paints and colours. We exclude these instances as well. These things also make our algorithm more generalizable because they deal with several different possible contexts of false positives that might pop up in everyday life.

##Implementation:
To implement this, we will make 3 lists. One list is for words that if come immediately before instance of silver or gold, we regard the article as false positive. The second is for words that if come immediately after gold or silver, then we regard the article as false positive. The final is for words which if they appear anywhere in the article, we regard it as false positive. By later adding or subtracting to these lists, we can make our algorithm more robust and generalizablle. An example of how the lists will be populated is that in the case of gold and silver being used as a part of a simili or a metaphor, they will probably have the word like or as immediately before them ('as valuable as gold'). Therefore, we add like and as to the list of words that come before. An important note is that this method can also be extended by adding more lists if that means better generalization. For example you can have different lists for gold and silver. This can help detect the silver lining false positive but prevent flagging a phrase like gold lining as false positive.

In [6]:
#Function that takes in an article and gives 0 if it does not contain 'gold' or 'silver' and 1 otherwise. Excludes False Positives.
def classify_excluding_false_positive(article,imediate_left_list,imediate_right_list,full_article_list):
    words = re.findall(r'\b\w+\b', article.lower())
    for i in range(len(full_article_list)):
      if full_article_list[i] in words:
        return 0
      else:
        continue
    if 'silver' in words or 'gold' in words:
      for i in range(len(words)):            #We look through all the words so that if gold or silver appears in two instances, both instances are checked
        if words[i]=='gold' or words[i]=='silver':
          if i==0:
            if words[i+1] in immediate_right_list:
              pass
            else:
              return 1                       #All the return 1 statements are there so that as soon as we find gold or silver being used in the right context, we label the article as one. This prevents the article from being labelled as false positive for example if silver is used in that context again later in the sentence.
          elif i==len(words)-1:
            if words[i-1] in immediate_left_list:
              pass
            else:
              return 1
          elif words[i-1] in immediate_left_list or words[i+1] in immediate_right_list:
            pass
          else:
            return 1
        else:
          continue
    return 0

In [7]:
immediate_left_list = ['as','like']
immediate_right_list = ['lining']
full_article_list = ['paint', 'color','colour']
sum = 0
for i in range(len(articles)):
  sum = sum+classify_excluding_false_positive(articles[i],immediate_left_list,immediate_right_list,full_article_list)
#print(classify_excluding_false_positive('Books are as valuable as gold',immediate_left_list,immediate_right_list,full_article_list))
print("Total articles containing silver or gold excluding false positives are: ", sum)

Total articles containing silver or gold excluding false positives are:  652


#Step 4 Accounting for false negatives
We account for false negatives by a simple extension of the function we made for false positives. The only difference is that we take in a list of aliases in our function for gold or silver. However, the important thing to not in implementation is that some of the aliases for gold and silvers are probably always used in trading environments (for example XAU/USD) so they will always refer to silver or gold as tradeable commodities. Therefore, there are some aliases that if present in the sentence should always be detected as silver or gold being used as tradeable commodities so they do not have the risk of being a false positive while other aliases that are just synonyms of gold and silver so they do have a risk of being a false positive. We handle this by passing to out function a list for trading environment aliases and a list for synonyms of gold and silver. Then we apply the false positive techniques only to the latter list. This helps us prevent missing the detection of sentences like 'I bought XAU/USD from my gold painted mobile phone.' Therefore, this makes our algorithm more generalizable.

In [8]:
#Function that takes in an article and gives 0 if it does not contain 'gold' or 'silver' or any of its aliases and 1 otherwise. Excludes False Positives and includes False negatives.
def classify_excluding_false_pos_neg(article,alias_trading,synonyms,imediate_left_list,imediate_right_list,full_article_list):
    words = re.findall(r'\b\w+\b', article.lower())
    if any(name in words for name in alias_trading):  #If any of the trading names of gold and silver are observed, we immediately output 1 because they are probably being used as tradeable commodities.
      return 1
    if any(name in words for name in full_article_list):
      return 0
    if any(name in words for name in synonyms):
      for i in range(len(words)):     #We look through all the words so that if gold or silver appears in two instances, both instances are checked
        if words[i] in synonyms:
          if i==0:
            if words[i+1] in immediate_right_list:
              pass
            else:
              return 1    #All the return 1 statements are there so that as soon as we find gold or silver being used in the right context, we label the article as one. This prevents the article from being labelled as false positive for example if silver is used in that context again later in the sentence.
          elif i==len(words)-1:
            if words[i-1] in immediate_left_list:
              pass
            else:
              return 1
          elif words[i-1] in immediate_left_list or words[i+1] in immediate_right_list:
            pass
          else:
            return 1
        else:
          continue
    return 0

In [9]:
alias_trading = ['xau','xag','xauusd','xagusd']
synonyms = ['ingot','ingots','gold','silver']
#print(classify_excluding_false_pos_neg('Lets buy XAU/USD from the market',alias_trading,synonyms,immediate_left_list,immediate_right_list,full_article_list))
sum = 0
for i in range(len(articles)):
  sum = sum+classify_excluding_false_pos_neg(articles[i],alias_trading,synonyms,immediate_left_list,immediate_right_list,full_article_list)
print("Total articles containing silver or gold or their aliases excluding false positives and false negatives are: ", sum)

Total articles containing silver or gold or their aliases excluding false positives and false negatives are:  656


## A Better Method:
Another method, which I believe can potentially be better than the above methods in detecting where silver and gold is being used as a tradeable commodity is to use a recurrent neural network. There are two main problems with our previous method. First, to see if the words gols and silver or their aliases are used as a tradeable commodity, there is a need to judge the overall context of the sentence. Our previous method only looks at the words to the immediate right or immediate left of the commodities. Moreover, for the cases where it looks at words throughout the article, it dismisses the article as a false positive without any contextual consideration. Therefore, our previous algorithm is very robotic. This can result in errors in detection. The second problem is that we cannot possible think of all the different contexts in which gold or silver or their alises are used as a tradeable commodity and we cannot think of all the false positive or false negative cases. However, the previous method hinges on us thinking of these instances and then puting the required words in the lists. Therefore, the previous method is not very generalizable. This is why we implement a recurrent neural network. Recurrent neural networks have the capability of taking in sequences of information and giving the final output while considering the sequence which is why it seems like a good idea.
## Implementation
In our implementation, we recognize that the first hurdle is that data is unlabelled. Therefore, we decide to label the data using Chat GPT. To do this, we first write the articles in a csv file and the articles column has a labels column next to it so Chat GPT can easily understand that we want to assign a label to each article.

In [10]:
# Path to the CSV file
csv_file = "articles_unlabelled.csv"

# Write the phrases to the CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Article","Label"])
    for article in articles:
        writer.writerow([article])

print("Articles have been written to", csv_file)

files.download('articles_unlabelled.csv')       #Used to download the file from Google Colab



Articles have been written to articles_unlabelled.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We explain to Chat GPT that we want to detect articles where silver and gold are mentioned as commodities. We tell it to label all such instances as 1 and other instances as 0. We also explain everything about false positive and false negatives and how it should label false positives and 0 and false negatives as 1 so that our RNN when trained on this data can recognize these instances and label them correctly. We expect ChatGPT to give better labelling results then our previous algorithm because it is an LLM and it is trained to detect context and sentiment in human language. However, after labelling, we will see that ChatGPT is still not as good as a human. In fact, it makes many mistakes. In general, during such a task one would always use human data labllers or human and AI hybrid data labelling to label the data. This was not done in this case because all this is just done as a proof of concept and there are resource and time limitations on the project. After getting the labelled data from ChatGPT, we read it, and make a vocabulary list of uniques words from it. From the vocabulary list, we will generate one-hot vectors for each word and allt he articles would be given to our RNN as a sequence of one-hot vectors representing the words in the articles. We ignore the punctuation marks. Our RNN itself takes in one-hot vectors and makes dense embeddings from them so that the vectors represent contxtual meaning as well. The RNN itself is designed such that it sequntially takes all the vectors and at the end of the article, provides log probabilities of the article belonging to one of the two classes: no silver/gold as commodities and silver/gold as commodities.

In [11]:
articles = []
labels = []
# Open the CSV file in read mode
with open('articles_labelled.csv', 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    ctr = 0
    # Iterate over each row in the CSV file
    for row in csv_reader:
        if(ctr==0):
          ctr = ctr+1
          continue
        # Each row is a list representing the columns in that row
        # Access elements in the row list using indexing
        if(row[0]==''):
          continue
        else:
          articles.append(row[0])
          labels.append(int(row[1]))


In [12]:

vocabulary_set = set()
for article in articles:
    # Basic preprocessing to remove punctuation and split by spaces
    words = re.findall(r'\b\w+\b', article.lower())
    vocabulary_set.update(words)

# Convert the set to a list and sort it to have a consistent order
vocabulary_list = sorted(list(vocabulary_set))

# Create a dictionary that maps words to indices
vocabulary = {word: idx for idx, word in enumerate(vocabulary_list)}



In [13]:
print(len(vocabulary))

18046


In [20]:
#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RNN(nn.Module):
    # implement RNN from scratch rather than using nn.RNN
    def __init__(self, input_size, embed_dim,hidden_size, output_size):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_size, embed_dim)     #used to create dense embedding from one-hot vecotrs
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(embed_dim + hidden_size, hidden_size)
        self.i2o = nn.Linear(embed_dim + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_tensor, hidden_tensor):
        embeded_tensor = self.embedding(input_tensor)
        if(len(embeded_tensor.size())==1):
          embeded_tensor = torch.unsqueeze(embeded_tensor,0)
        combined = torch.cat((embeded_tensor, hidden_tensor), 1)

        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return (torch.zeros(1, self.hidden_size)).to(device)     #initializes the hidden layer output for the first word
all_categories = [0,1]
n_categories = 2
n_hidden = 128
embed_dimension = 100    #one hot vectors of dimension 18046 (length of vocabulary list) converted to dense vectors of dimension 100
rnn = RNN(len(vocabulary), embed_dimension, n_hidden, n_categories)
rnn = rnn.to(device)


'''def category_from_output(output):
    category_idx = torch.argmax(output).item()
    return all_categories[category_idx]'''


criterion = nn.NLLLoss()
learning_rate = 0.005
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

def train(article_tensor, category_tensor):    #The functions runs a training step for a single article and its category
    hidden = rnn.init_hidden()
    for i in range(len(article_tensor)):
      output,hidden = rnn(article_tensor[i],hidden)
    loss = criterion(output, category_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return output, loss.item()

def select_article(articles,index,labels):    #The function is used to select article at a particular index and convert it into appropriate form and return it and its corresponding label
  wrds = re.findall(r'\b\w+\b', articles[index].lower())
  wrds_idx = []
  for j in range(len(wrds)):
    wrds_idx.append(vocabulary.get(wrds[j]))
  return (torch.tensor(wrds_idx)).to(device),(torch.tensor([labels[counter]])).to(device)

def check_accuracy(article_arr,label_arr,model):   #Checks the accuracy of the trained rnn model on the entire dataset
  sum_correct = 0
  for i in range(len(article_arr)):
    indices,category = select_article(article_arr,i,label_arr)
    with torch.no_grad():
      hidden = model.init_hidden()
      for j in range(len(indices)):
        output,hidden = model(indices[j],hidden)
      pred_cat = torch.argmax(output)
      if(pred_cat==label_arr[i]):
        sum_correct+=1
  return (sum_correct/len(article_arr))*100





current_loss = 0
all_losses = []
print_steps = 25
n_iters = 900
counter = 0

for i in range(n_iters):    #This helps loop through our list of articles
    wrds_idx,category = select_article(articles,counter,labels)     #takes a particular article at a particular index, converts it into a suitable format so it can be fed into our rnn
    counter = (counter+1)%len(labels)    #prevents counter from going out of range of the list of articles
    if(len(wrds_idx)==0):
      continue
    output, loss = train(wrds_idx, category)
    current_loss += loss

    if (i+1) % print_steps == 0:        #prints some training statistics every few steps
      print("The loss is: ",current_loss)
      acc = check_accuracy(articles,labels,rnn)
      print("The accuracy is: ",acc)
      print("The step is: ", i+1)
      current_loss = 0

The loss is:  14.70780636370182
The accuracy is:  56.470588235294116
The step is:  25
The loss is:  14.806015871465206
The accuracy is:  55.42483660130719
The step is:  50
The loss is:  18.625802136957645
The accuracy is:  55.751633986928105
The step is:  75
The loss is:  9.636103849858046
The accuracy is:  55.032679738562095
The step is:  100
The loss is:  10.896702287718654
The accuracy is:  55.947712418300654
The step is:  125
The loss is:  12.536292105913162
The accuracy is:  54.90196078431373
The step is:  150
The loss is:  8.461463619023561
The accuracy is:  54.18300653594771
The step is:  175
The loss is:  11.680743781849742
The accuracy is:  52.28758169934641
The step is:  200
The loss is:  7.644193306565285
The accuracy is:  52.54901960784314
The step is:  225
The loss is:  7.675816901959479
The accuracy is:  51.83006535947713
The step is:  250
The loss is:  8.188517524395138
The accuracy is:  51.37254901960784
The step is:  275
The loss is:  4.00129905808717
The accuracy is: 

In [15]:
def predict(article,vocabulary,model):       #After training, this function can be give any article. It can then output its prediction regarding that article.
  words = re.findall(r'\b\w+\b', article.lower())
  indices = []
  for i in range(len(words)):
    if (vocabulary.get(words[i])==None):
      continue
    else:
      indices.append(vocabulary.get(words[i]))
  indices = (torch.tensor(indices)).to(device)
  hidden = model.init_hidden()
  for i in range(len(indices)):
    output,hidden = model(indices[i],hidden)
  pred_cat = torch.argmax(output)
  if(pred_cat==0):
    print('The string does not contain the words gold or silver being referred to as a tradeable commodity')
  else:
    print('The string contains either gold or silver being referred to as a tradeable commodity')


Here it is important to not that there are several ways in which the training process of the model can be improved. They are listed below:
For example, the first thing I would do is to chage the code a bit such that

1.   As mentioned before, use human data labellers for better model performance on every day speech.
2.   Change the code a bit so model can be trained on batched data to reap the benefits of parallel processing capabilities of pytorch. This would help reduce the training time. For now, a trained state of the model has been provided and it is loaded in the next cell so the person evaluating this notebook can just run the next cell without going through all the above training steps and evaluate the model and its performance easily and play aroung with it.
3.   Split the data into training, validation and testing data. Tune hyperparameters using validation data and evaluate accuracy using testing data to check how well the model performs in general.
4.   We see that in our training loop, the number of iterations are so low they do not even go through the entire data and the accuracy is only about 70%. This is because the gradients exploded in later iterations. This can be avoided easily by experimenting around with other optimizers (for example SGD can be used with weight decay and momentum or Adam optimizer can be used). If this is done then the neural network can achieve much higher accuracies.

The reason why all this has not been done is due to time and resource constraints. At the end of the day, this is not intended to be a final product but it is just there to show the evaluator what my conception of a much better solution than the one above would be.

In [ ]:
torch.save(rnn.state_dict(),'rnn.pth')    #Used to save the trained model

In [16]:
#This code block and the immediate next one can be run without running the previous training code block because they use the saved model provided with the jupyter notebook. Intended to save time of the evaluator.
#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class RNN(nn.Module):
    # implement RNN from scratch rather than using nn.RNN
    def __init__(self, input_size, embed_dim,hidden_size, output_size):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_size, embed_dim)     #used to create dense embedding from one-hot vecotrs
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(embed_dim + hidden_size, hidden_size)
        self.i2o = nn.Linear(embed_dim + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_tensor, hidden_tensor):
        embeded_tensor = self.embedding(input_tensor)
        if(len(embeded_tensor.size())==1):
          embeded_tensor = torch.unsqueeze(embeded_tensor,0)
        combined = torch.cat((embeded_tensor, hidden_tensor), 1)

        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return (torch.zeros(1, self.hidden_size)).to(device)     #initializes the hidden layer output for the first word
n_categories = 2
n_hidden = 128
embed_dimension = 100
rnn_loaded = RNN(len(vocabulary), embed_dimension, n_hidden, n_categories)
rnn_loaded.load_state_dict(torch.load('rnn.pth',map_location=torch.device(device)))
print('The accuracy of this model is: ', check_accuracy(articles,labels,rnn_loaded))



The accuracy of this model is:  70.84967320261438


In [18]:
#Can be used to play around with the saved model
predict('I just bought gold. I am happy.', vocabulary, rnn_loaded)
predict('The silver lining to all the tough times was that I became a better person',vocabulary,rnn_loaded)
predict('I am going up the mountain.',vocabulary,rnn_loaded)

The string contains either gold or silver being referred to as a tradeable commodity
The string does not contain the words gold or silver being referred to as a tradeable commodity
The string does not contain the words gold or silver being referred to as a tradeable commodity


##Step 5 Sentiment Analysis:
To do this, the strategy we use is that we make a dictionary of negative sentiment and positive sentiment words. For every article, we keep a sentiment score and we increment the score if positive sentiment words are found and decrement the score if negative sentiment words are found. At the end, we make 4 classes according to sentiment score. For example if the sentiment score is very high, we conclude that the article has very positive sentiment. This not only tells us the sentiment represented in the articles but also how positive or negative the sentiment is. If sentiment is very positive, we can be assured that the article reflects positive expectations from gold or silver. The limitations of this method again is first that it cannot possible assess the context accurately in general situations. An example is 'We expect Apple stock to go up but gold to go down.' Our method would flag this as neutral but actually it indicates negative sentiment towards gold. One way to prevent this can be to assign higher sentiment score when words are closer to 'gold' or 'silver' but this too is not very generalizable. Moreover, we also cannot possibly type up a python dictionary containing all the negative and positive sentiment words. To counter these problems, we can again use a RNN very similar to the one in the previous problem. The only difference would be the labelling of data and that the output of RNN would be of size 5 representing the 5 classes: very positive, positve, neutral, very negative, negative. Due to the advantages of RNNs described above, given enough data, the RNN would capture the complicated contexts and sentiments of human language much better than any hard coded dictionary.

In [19]:
articles_gold = []    #Seperate articles that mention gold or silver in the context of being a tradeable commodity according to our first algorithms
for i in range(len(articles)):
  if classify_excluding_false_pos_neg(articles[i],alias_trading,synonyms,immediate_left_list,immediate_right_list,full_article_list)==1:
    articles_gold.append(articles[i])
sentiment_dict = {    #We plan to maintain a sentiment score. Negetive words decrease the score and positive words increase the score.
    'rise': 1,
    'gain': 1,
    'upward': 1,
    'bullish': 1,
    'profit': 1,
    'surge': 1,
    'increase': 1,
    'growth': 1,
    'positive': 1,
    'improve': 1,
    'rally': 1,
    'fall': -1,
    'drop': -1,
    'downward': -1,
    'bearish': -1,
    'loss': -1,
    'decline': -1,
    'decrease': -1,
    'slump': -1,
    'negative': -1,
    'dip': -1,
    'crash': -1
}
neutral = 0
positive = 0
negative = 0
very_positive = 0
very_negative = 0
for i in range(len(articles_gold)):
  words = re.findall(r'\b\w+\b', articles_gold[i].lower())
  sentiment_score = 0
  for word in words:
    if word in sentiment_dict.keys():
      sentiment_score = sentiment_dict.get(word) + sentiment_score
  if(sentiment_score>1):     #Sepereation of articles according to 4 different sentiment scores.
    very_positive+=1
  elif(sentiment_score<-1):
    very_negative+=1
  elif(sentiment_score==-1):
    negative+=1
  elif(sentiment_score==0):
    neutral+=1
  elif(sentiment_score==1):
    positive+=1
print('The number of aritcles where the sentiment is very positive are: ', very_positive)
print('The number of aritcles where the sentiment is positive are: ', positive)
print('The number of aritcles where the sentiment is neutral are: ', neutral)
print('The number of aritcles where the sentiment is negative are: ', negative)
print('The number of aritcles where the sentiment is very negative are: ', very_negative)






The number of aritcles where the sentiment is very positive are:  27
The number of aritcles where the sentiment is positive are:  13
The number of aritcles where the sentiment is neutral are:  582
The number of aritcles where the sentiment is negative are:  21
The number of aritcles where the sentiment is very negative are:  13


#Aliases Task
To find reference to Broadcom Inc since 2008 from any given textual data, the first step is to conduct research on the different name changes that the company went through. To do this, we use Bing chat and explain to it that some places where name changes might be found are mergers, demergers, acquisitions or restructurings to get better results. Moreover, we search the web as well for more information. The reason we use Bing Chat instead of Chat GPT for this is that Bing Chat has web search capabilities so it can provide latest results while Chat GPT has a limitation on how much recent information it has due to its dataset and the fact that it does not have capabilities to search the internet.
##Next Steps:
Infromation gained from our research tells us that the roots of Broadcom Inc can be traced back to a company called Avago Technologies. This company existed from 2005 to 2016 so it is witin the time window that we are interested. In 2016, Avago Technologies merged with Broadcom Corporation to form Broadcom Limited. This would later change its name to form the present Broadcom Inc. Therefore, Broadcom Inc. has its roots in both Avago Technologies and Broadcom Corporation since in 2008-2016, it existed as two seperate companies that were later merged. Since we want to track mentions of Broadcom Inc it also makes sense to track mentions of Avago Technologies and Broadcom Corporation because Broadcom Inc now encapsulates both these companies. Therefore, we can simply make a python list containing the names 'Avago Technologies' and 'Broadcom Corporation' and 'Broadcom Inc', which are all the names that the company has taken since 2008. We also include 'BRCM' in the list since Broadcom Corporation was a publicly traded company with this ticker symbol. Finally, we include 'AVGO' since that is the current ticker symbol for Broadcom Inc. Avago Technologies does have previos name change history but it took place in 2005 so it is outside the scope of our considerantion. Broadcom Corporation does not have any other significant name change history before the merger otherwise we might have had to add more names to the list. After finalizing the list using the research we conducted, we just look for names in the list trough our data to look for mentions of Broadcom Inc since 2008 just like we did with 'gold' and 'silver'.  